In [11]:
import pandas as pd

In [12]:
def get_java_file_name(v):
	x = v.split('/')
	y = x[-1].split('#')
	javaFileName = y[0]
	return javaFileName

def get_comment_line(v):
	x = v.split('/')
	y = x[-1].split('#')
	line = y[1][1:]
	return int(line)

def get_code_from_javadoc_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        if not (comment.startswith('/**') and comment.endswith('*/')):
            while comment != '*/':
                comment = lines[commentStartLine + i].strip()
                i += 1
        firstCodeLine = lines[commentStartLine + i].strip()
    return firstCodeLine

def get_code_from_line_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        isComment = comment.startswith('//')
        if not isComment:
            code = comment.split('//')
            firstCodeLine = code[0].strip()
        else:
            while isComment:
                if 0 <= commentStartLine + i < len(lines):
                    comment = lines[commentStartLine + i].strip()
                    isComment = comment.startswith('//')
                    if isComment:
                        i += 1
                else:
                    isComment = False
            if 0 <= commentStartLine + i < len(lines):
                firstCodeLine = lines[commentStartLine + i].strip()
            else:
                firstCodeLine = ''
    return firstCodeLine 
    
def get_code_from_block_comment(lines, commentStartLine):
    firstCodeLine = ''
    if 0 <= commentStartLine < len(lines):
        comment = lines[commentStartLine].strip()
        i = 1
        startsWithComment = comment.startswith('/*')
        if not startsWithComment:
            code = comment.split('/*')
            firstCodeLine = code[0].strip()
        else:
            while comment != '*/':
                comment = lines[commentStartLine + i].strip()
                i += 1
            firstCodeLine = lines[commentStartLine + i].strip()
    return firstCodeLine

## extract One line code after comment
## Outfile: Path to output CSV
## Javadir: dir of Java files
## csvPath: csv Database from project
def extractCode(Outfile,Javadir,csvPath):
    dataframe = pd.read_csv(csvPath, na_filter = False)
    #names = ['id', 'type', 'comment', 'code','non-information']
    names = ['id', 'type', 'comment', 'code','Expected']
    df = pd.DataFrame()
    for i,data in dataframe.iterrows():
#         print(i)
        link = data["link_to_comment"]
        file_type = data["type"]
        javaFileName = get_java_file_name(link)
        line = get_comment_line(link)

        f = open(Javadir+"/" + javaFileName, "r")
        lines = f.readlines()
        line = line - 1
        code = 'undefined type'
        if file_type == 'Javadoc':
            code = get_code_from_javadoc_comment(lines, line)
        elif file_type == 'Line':
            code = get_code_from_line_comment(lines, line)
        elif file_type == 'Block':
            code = get_code_from_block_comment(lines, line)

    #     print("Code:",code)
    #     print()
        df = df.append(pd.DataFrame([[data["ID"],data["type"],data["comment"],code,data["Expected"]]], columns=names))
        f.close()
#     print(df)
    df.to_csv(Outfile, index=False)

In [13]:
#
Outfile ="csv/oneLineCode3.csv"
Javadir ="data/"
csvPath = "assets/test_set_0520.csv"
extractCode(Outfile,Javadir,csvPath)

In [14]:
#
csvPath = "csv/oneLineCode3.csv"
dataframe = pd.read_csv(csvPath, na_filter = False)
keep_col = ['comment','code','Expected']
new_f = dataframe[keep_col]
new_f.to_csv("csv/newFile3.csv", index=False)
